<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Generative_Adversarial_Network_(GAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class Generator(nn.Module):
    def __init__(self, noise_dim, image_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(),
            nn.Linear(128, image_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self, image_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(image_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.fc(x)

# Initialize models and hyperparameters
noise_dim = 100
image_dim = 28 * 28
G = Generator(noise_dim, image_dim)
D = Discriminator(image_dim)

criterion = nn.BCELoss()
G_optimizer = optim.Adam(G.parameters(), lr=0.0002)
D_optimizer = optim.Adam(D.parameters(), lr=0.0002)

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
batch_size = 64

# GAN training loop (simplified)
for epoch in range(10):
    for real_images, _ in dataloader:
        # Train Discriminator on real and fake data
        D_optimizer.zero_grad()
        real_images = real_images.view(real_images.size(0), -1)  # Flatten the images
        real_labels = torch.ones(real_images.size(0), 1)
        fake_labels = torch.zeros(real_images.size(0), 1)
        real_output = D(real_images)
        real_loss = criterion(real_output, real_labels)

        noise = torch.randn(real_images.size(0), noise_dim)
        fake_images = G(noise)
        fake_output = D(fake_images.detach())
        fake_loss = criterion(fake_output, fake_labels)
        D_loss = real_loss + fake_loss
        D_loss.backward()
        D_optimizer.step()

        # Train Generator
        G_optimizer.zero_grad()
        fake_output = D(fake_images)
        G_loss = criterion(fake_output, real_labels)
        G_loss.backward()
        G_optimizer.step()

    print(f"Epoch [{epoch+1}/10], D Loss: {D_loss.item():.4f}, G Loss: {G_loss.item():.4f}")